# Import Library

In [1]:
import pandas as pd
import numpy as np
import torch
import os
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Data

In [3]:
features_df = pd.read_csv('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/DataSplitSW/data_job_sw.csv')
labels_df = pd.read_csv('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/DataSplitSW/salary_sw.csv')

In [4]:
features_df.head()

,Unnamed: 0,company_name,location,category_1,category_2,salary_min,title_translated,cleaned_description_with_stopwordremove_translated,type_translated,role_translated
0,2093,samudra toys indonesia,jakarta raya,penjualan,manajemen akun dan relasi,5500000,Alat Tulisan Tokoh Eksekutif Akun Kunci,cv samudra toy indonesia usaha importir distri...,penuh waktu,Eksekutif Akun Utama
1,109,aia financial,banten,perbankan dan layanan finansial,layanan klien,7000000,Konsultan Kehidupan Bancassurance Pertama Serang,tugas tanggung jawab laku jual produk asuransi...,penuh waktu,Konsultan Kehidupan Bancassurance
2,4255,tirepro,jakarta raya,teknik,teknik otomotif,4000000,Mecanik Mobil,lokasi jakarta selatan pusat utara barat sedia...,Contrak,mekanik mobil
3,5392,xianghui trading indonesia,jakarta raya,periklanan,seni dan media,4500000,Pembuat konten,kualifikasi lakilaki prempuan alam bidang cont...,Error: 'Translator' object has no attribute 'r...,Pembuat konten
4,5093,aexel bio medis,jakarta raya,penjualan,perwakilanataukonsultan penjualan,6000000,Penjualan Eksekutif,Eksekutif Penjualan Membangun Kembangkan Perta...,penuh waktu,Penjualan Eksekutif


In [5]:
labels_df.head()

,salary_min
0,5500000
1,7000000
2,4000000
3,4500000
4,6000000


In [6]:
features_df = features_df.drop(columns=['Unnamed: 0'])

In [7]:
features_df.head()

,company_name,location,category_1,category_2,salary_min,title_translated,cleaned_description_with_stopwordremove_translated,type_translated,role_translated
0,samudra toys indonesia,jakarta raya,penjualan,manajemen akun dan relasi,5500000,Alat Tulisan Tokoh Eksekutif Akun Kunci,cv samudra toy indonesia usaha importir distri...,penuh waktu,Eksekutif Akun Utama
1,aia financial,banten,perbankan dan layanan finansial,layanan klien,7000000,Konsultan Kehidupan Bancassurance Pertama Serang,tugas tanggung jawab laku jual produk asuransi...,penuh waktu,Konsultan Kehidupan Bancassurance
2,tirepro,jakarta raya,teknik,teknik otomotif,4000000,Mecanik Mobil,lokasi jakarta selatan pusat utara barat sedia...,Contrak,mekanik mobil
3,xianghui trading indonesia,jakarta raya,periklanan,seni dan media,4500000,Pembuat konten,kualifikasi lakilaki prempuan alam bidang cont...,Error: 'Translator' object has no attribute 'r...,Pembuat konten
4,aexel bio medis,jakarta raya,penjualan,perwakilanataukonsultan penjualan,6000000,Penjualan Eksekutif,Eksekutif Penjualan Membangun Kembangkan Perta...,penuh waktu,Penjualan Eksekutif


# Cross Check Data

In [8]:
# Periksa apakah ada nilai yang hilang
print("Missing values in features dataset:")
print(features_df.isnull().sum())
print("Missing values in labels dataset:")
print(labels_df.isnull().sum())

Missing values in features dataset:
company_name                                          0
location                                              0
category_1                                            0
category_2                                            0
salary_min                                            0
title_translated                                      0
cleaned_description_with_stopwordremove_translated    0
type_translated                                       0
role_translated                                       0
dtype: int64
Missing values in labels dataset:
salary_min    0
dtype: int64


In [9]:
features_df.dropna(inplace=True)
labels_df.dropna(inplace=True)

In [10]:
# Drop unused columns and prepare data
labels = labels_df['salary_min'].values
texts = features_df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [11]:
texts

,0
0,samudra toys indonesia jakarta raya penjualan ...
1,aia financial banten perbankan dan layanan fin...
2,tirepro jakarta raya teknik teknik otomotif 4...
3,xianghui trading indonesia jakarta raya perikl...
4,aexel bio medis jakarta raya penjualan perwak...
...,...
4846,ofero technology indonesia jakarta raya pemas...
4847,jepha center banten pendidikan dan pelatihan ...
4848,tata asri sekawan leoni gelato bali administra...
4849,berkat sumber rizki jakarta raya penjualan pe...


In [12]:
labels

array([5500000, 7000000, 4000000, ..., 3500000, 9000000, 2500000])

In [13]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4851 entries, 0 to 4850
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   salary_min  4851 non-null   int64
dtypes: int64(1)
memory usage: 38.0 KB


# Tokenize

Kelas TextDataset dirancang untuk mempersiapkan data teks dan label menjadi format yang sesuai untuk model, dengan melakukan tokenisasi menggunakan BertTokenizer dan memastikan panjang teks konsisten melalui padding dan truncation. Fungsi __getitem__ mengembalikan input IDs, attention mask, dan label dalam bentuk tensor, yang diperlukan untuk melatih model BERT. Data dibagi menjadi train dan test set menggunakan train_test_split untuk evaluasi performa model secara terpisah.

In [14]:
# 2. Tokenization with BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Definisi kelas dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    # Ambil jumlah sampel dalam dataset
    def __len__(self):
        return len(self.texts)

    # Ambil teks dan label berdasarkan indeks
    def __getitem__(self, index):
        text = self.texts.iloc[index] if isinstance(self.texts, pd.Series) else self.texts[index]
        label = self.labels.iloc[index] if isinstance(self.labels, pd.Series) else self.labels[index]

        # Tokenisasi teks menggunakan Bert
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )


        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float)
        }

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Normalisasi

melakukan normalisasi nilai label menggunakan MinMaxScaler agar skala data lebih seragam, penting untuk stabilitas pelatihan model. Dataset pelatihan dan pengujian dibuat menggunakan kelas TextDataset, lalu dibungkus dalam DataLoader untuk menyediakan data secara batch dan mendukung shuffling pada data pelatihan. DataLoader mempermudah pembacaan data dalam jumlah besar selama pelatihan dengan memuat input IDs, attention mask, dan label dalam format tensor. Perulangan di akhir mencetak dimensi data dalam satu batch, memastikan format data sesuai untuk dimasukkan ke model.

jgn lupa buat denormalisasinya nanti

In [ ]:
# scaler = MinMaxScaler()

# train_labels = scaler.fit_transform(train_labels.reshape(-1, 1)).squeeze()
# test_labels = scaler.transform(test_labels.reshape(-1, 1)).squeeze()

In [16]:
# Menormalisasi data target (train_labels dan test_labels)
scaler = MinMaxScaler()
train_labels = scaler.fit_transform(train_labels.reshape(-1, 1)).squeeze()
test_labels = scaler.transform(test_labels.reshape(-1, 1)).squeeze()

# Menyimpan scaler agar bisa digunakan pada prediksi di kemudian hari
joblib.dump(scaler, '/content/drive/MyDrive/6. DEEP LEARNING/DataFix/scaler_baru.pkl')

['/content/drive/MyDrive/6. DEEP LEARNING/DataFix/scaler_baru.pkl']

In [17]:
# Membuat objek dataset
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

# Membuat loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [18]:
for batch in train_loader:
    print(f"Input IDs shape: {batch['input_ids'].shape}")
    print(f"Attention Mask shape: {batch['attention_mask'].shape}")
    print(f"Label shape: {batch['label'].shape}")
    break

Input IDs shape: torch.Size([16, 128])
Attention Mask shape: torch.Size([16, 128])
Label shape: torch.Size([16])


# Modeling

model BERT + LSTM untuk memanfaatkan kekuatan representasi kontekstual BERT dan kemampuan LSTM menangkap hubungan sekuensial. Model menggunakan BERT untuk menghasilkan representasi fitur dari teks, yang diteruskan ke LSTM bidirectional untuk memproses informasi temporal, dan akhirnya melalui lapisan fully connected (fc) untuk prediksi nilai tunggal. Fungsi forward mendefinisikan alur data dari token input ke prediksi model. Model diinisialisasi dengan mean squared error loss (MSE) untuk mengukur kesalahan regresi dan dioptimalkan menggunakan algoritma Adam dengan learning rate 2e-5.

In [19]:
# 3. Model definition: BERT + LSTM
class BertLSTMModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', lstm_hidden_size=128, lstm_layers=1):
        super(BertLSTMModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name) # Model Bert
        self.lstm = nn.LSTM(                                   # Model LSTM
            input_size=self.bert.config.hidden_size,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )
        # Untuk mengubah output menjadi prediksi nilai tunggal
        self.fc = nn.Linear(lstm_hidden_size * 2, 1)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # [CLS] token
        cls_embeddings = bert_output.last_hidden_state[:, 0, :]
        lstm_output, _ = self.lstm(cls_embeddings.unsqueeze(1))
        output = self.fc(lstm_output[:, -1, :])
        return output

# Instantiate model, loss, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertLSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Modeling yang dilakukan menggunakan Bert dan LSTM. Hidden layers yang digunakan pada LSTM adalah 128 dengan LSTM layer sebanyak 1.

# Train Model

fungsi train_model untuk melatih model BERT + LSTM menggunakan data train dan mengevaluasi performa pada data test. Dalam setiap epoch, model dilatih dengan memproses batches dari train_loader, menghitung loss dengan criterion (MSE), dan memperbarui bobot menggunakan optimizer. Setelah pelatihan, model dievaluasi dengan test_loader dalam mode evaluasi (model.eval()), menghitung rata-rata loss pada data uji untuk menilai generalisasi model. Fungsi ini memastikan pelatihan model terkelola dengan baik dan memonitor performa secara bertahap melalui metrik loss.

In [20]:
# 4. Training Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=15):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        # Iterasi batch
        for batch in train_loader:
            # Memandahkan data ke GPU agar dapat di proses lebih cepat
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)

            # Untuk menghitung loss
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            # Akumulasi total loss
            total_loss += loss.item()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}')

        # Evaluasi data test
        model.eval()
        test_loss = 0
        with torch.no_grad():
            # Iterasi batch
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs.squeeze(), labels)
                test_loss += loss.item()

        print(f'Test Loss: {test_loss / len(test_loader):.4f}')

In [21]:
# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer)

Epoch 1/15, Loss: 0.0017
Test Loss: 0.0012
Epoch 2/15, Loss: 0.0014
Test Loss: 0.0012
Epoch 3/15, Loss: 0.0013
Test Loss: 0.0010
Epoch 4/15, Loss: 0.0012
Test Loss: 0.0010
Epoch 5/15, Loss: 0.0011
Test Loss: 0.0009
Epoch 6/15, Loss: 0.0010
Test Loss: 0.0008
Epoch 7/15, Loss: 0.0009
Test Loss: 0.0012
Epoch 8/15, Loss: 0.0008
Test Loss: 0.0009
Epoch 9/15, Loss: 0.0008
Test Loss: 0.0007
Epoch 10/15, Loss: 0.0007
Test Loss: 0.0008
Epoch 11/15, Loss: 0.0006
Test Loss: 0.0011
Epoch 12/15, Loss: 0.0006
Test Loss: 0.0006
Epoch 13/15, Loss: 0.0005
Test Loss: 0.0014
Epoch 14/15, Loss: 0.0005
Test Loss: 0.0004
Epoch 15/15, Loss: 0.0004
Test Loss: 0.0005


Model diatas tidak overfitting karena hasil testing lebiih kecil dibandingkan dengan hasil training loss.


# Evaluation

In [22]:
# Prediksi pada data uji
model.eval()
with torch.no_grad():
    preds = []
    true_labels = []
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask)
        preds.extend(outputs.squeeze().cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Hitung MAE
mae = mean_absolute_error(true_labels, preds)
print(f'Mean Absolute Error: {mae:.4f}')

Mean Absolute Error: 0.0120


Nilai **Mean Absolute Error (MAE)** sebesar 0.0173 menunjukkan bahwa rata-rata kesalahan prediksi model terhadap nilai sebenarnya sangat kecil, yang menandakan performa regresi yang baik. Hasil ini menunjukkan model mampu menghasilkan prediksi yang cukup akurat untuk tugas yang diberikan.

In [23]:
# Denormalisasi hasil prediksi
preds_denorm = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).squeeze()

untuk menginterpretasikan hasil prediksi dalam konteks nilai sebenarnya, sehingga lebih bermakna dan dapat dibandingkan langsung dengan data asli.

# Save Model

In [24]:
# Menyimpan model
torch.save(model.state_dict(), '/content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_full_baru.pth')

# Memuat model
model = BertLSTMModel()
model.load_state_dict(torch.load('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_full_baru.pth'))
model.eval()

<ipython-input-24-52c5e0eaf447>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_

BertLSTMModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

# Testing Model

## Memuat model yang telah di save

In [25]:
import pandas as pd
import numpy as np
import torch
import os
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
import joblib

In [26]:
class BertLSTMModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', lstm_hidden_size=128, lstm_layers=1):
        super(BertLSTMModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name) # Model Bert
        self.lstm = nn.LSTM(                                   # Model LSTM
            input_size=self.bert.config.hidden_size,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )
        # Untuk mengubah output menjadi prediksi nilai tunggal
        self.fc = nn.Linear(lstm_hidden_size * 2, 1)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # [CLS] token
        cls_embeddings = bert_output.last_hidden_state[:, 0, :]
        lstm_output, _ = self.lstm(cls_embeddings.unsqueeze(1))
        output = self.fc(lstm_output[:, -1, :])
        return output

In [38]:
# Mount Google Drive
drive.mount('/content/drive')

# Load the scaler and tokenizer
scaler = joblib.load('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/scaler_sw.pkl')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the full model (architecture + weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Create an instance of your model class
model = BertLSTMModel()  # Assuming BertLSTMModel is defined as in your previous code

# 2. Load the state dictionary into the model instance
state_dict = torch.load('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_full_baru.pth', map_location=device)
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

print(f"Model telah dimuat dari /content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_full_baru.pth")

# Now you can directly use the model for inference

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-38-e83a55d2f4a9>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_fu

Model telah dimuat dari /content/drive/MyDrive/6. DEEP LEARNING/DataFix/bert_lstm_model_full_baru.pth


## Uji Model

In [41]:
# Fungsi untuk prediksi data baru
def predict_new_data(model, text_input, tokenizer, scaler, max_len=128):
    # Tokenisasi dan inferensi seperti biasa
    encoding = tokenizer(text_input, truncation=True, max_length=max_len, padding='max_length', return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)  # Memindahkan input_ids ke device
    attention_mask = encoding['attention_mask'].to(device)  # Memindahkan attention_mask ke device

    # Pastikan model juga berada di device yang sama
    model.to(device)  # Memindahkan model ke device

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        pred = output.squeeze().cpu().numpy()  # Hasil prediksi pada skala normalisasi

    # Denormalisasi hasil prediksi
    pred_denorm = scaler.inverse_transform(np.array(pred).reshape(-1, 1)).squeeze()

    return pred_denorm

In [42]:
new_text = "Melakukan analisis statistik dan data mining untuk mengidentifikasi pola dan tren"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 4656043.00


In [43]:
new_text = "pt mitracomm ekasarana sedang buka lowong kerja dengan posisi agent digital cimanggis depok requirement priaatauwanita usia mak th pendidkan min dataus jurus ilmu komunikasi management sastra inggris ilmu komunikasi milik paham tentang social medium milik alam di customer service telcoataudigital sosmedatau call center min tahun mampu operasi m office milik mampu bahasa inggris sedia shifting benefit jenjang karir gaji tetap tunjang bpjs sehat bpjs ketenagakerjaan"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 2856240.00


In [44]:
new_text = "data engineer jakarta wfo"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 3545409.00


In [45]:
new_text = "bio tosca estetika jakarta raya akuntansi akuntansi dan pelaporan finansial Analis Pembelian Keuangan didik minimal diploma sarjana akuntansi manajemen ekonomi teknik industri teknik kimia ekuivalen ipk minimal skala mampu operasi microsoft office utama excel milik paham kena prose milik integritas teliti tekun adaptif tanggung jawab milik pribadi baik disiplin akurat mandiri kerja tim tampil baik rapi maksimal usia tahun milik ajar cekat utama milik alam min tahun penuh waktu analis membeli"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 5202359.50


In [47]:
new_text = "angkasa surya abadi jawa timur konstruksi manajemen proyek Konstruksi Manajer Proyek kualifikasi milik paham baik hadap metode planning technical document problem solving bidang konstruksi milik jiwa leadership baik atur kerjasama tim koordinasi rekan kerja baik detail oriented milik mampu komunikasi baik fluent bahasa inggris monitor jalan suatu proyek tempat kantor surabaya Contrak Manajer Konstruksi"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 6358562.50


In [ ]:
testing = pd.read_csv('/content/drive/MyDrive/6. DEEP LEARNING/DataFix/DataSplitSW/data_test_yoni.csv')

testing.head()

,new_text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,clinic viva medika jawa barat pemasaran dan ko...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sahabat jaya sukses jakarta raya pemasaran dan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,maxima daya indonesia sumatera selatan konstru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,intersolusi teknologi asia jawa barat teknolog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,orang tua group jawa timur penjualan perwakila...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
testing = testing[['new_text']]

In [ ]:
testing.head()

,new_text
0,clinic viva medika jawa barat pemasaran dan ko...
1,sahabat jaya sukses jakarta raya pemasaran dan...
2,maxima daya indonesia sumatera selatan konstru...
3,intersolusi teknologi asia jawa barat teknolog...
4,orang tua group jawa timur penjualan perwakila...


In [ ]:
# Lakukan prediksi untuk semua baris pada kolom new_text
testing['predicted_salary'] = testing['new_text'].apply(
    lambda text: predict_new_data(model, text, tokenizer, scaler)
)

# Simpan hasil ke file CSV
output_file = '/content/drive/MyDrive/6. DEEP LEARNING/DataFix/DataSplitSW/predicted_salaries.csv'
testing.to_csv(output_file, index=False)
print(f"Hasil prediksi disimpan ke file: {output_file}")

Hasil prediksi disimpan ke file: /content/drive/MyDrive/6. DEEP LEARNING/DataFix/DataSplitSW/predicted_salaries.csv


# New

In [ ]:
# Fungsi untuk prediksi data baru
def predict_new_data(model, text_input, tokenizer, scaler, max_len=128):
    # Tokenisasi dan inferensi seperti biasa
    encoding = tokenizer(text_input, truncation=True, max_length=max_len, padding='max_length', return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        pred = output.squeeze().cpu().numpy()  # Hasil prediksi pada skala normalisasi

    # Denormalisasi hasil prediksi
    pred_denorm = scaler.inverse_transform(np.array(pred).reshape(-1, 1)).squeeze()

    return pred_denorm

# Contoh pengujian
new_text = "Membantu dokter dalam melayani customer menjalankan fungsi perawat/ asisten dokter dengan baik Bali"
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")


Prediksi gaji untuk input baru: 3794916.25


In [ ]:
# Fungsi untuk prediksi data baru
def predict_new_data(model, text_input, tokenizer, scaler, max_len=128):
    # Tokenisasi dan inferensi seperti biasa
    encoding = tokenizer(text_input, truncation=True, max_length=max_len, padding='max_length', return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        pred = output.squeeze().cpu().numpy()  # Hasil prediksi pada skala normalisasi

    # Denormalisasi hasil prediksi
    pred_denorm = scaler.inverse_transform(np.array(pred).reshape(-1, 1)).squeeze()

    return pred_denorm

# Contoh pengujian
new_text = "berpengalaman minimal 2 tahun sebagai sales dalam bidang Education S1 Memenuhi target yang telah dicanangkan oleh supervisor/atasan."
predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)
print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")


Prediksi gaji untuk input baru: 6851490.50


Dibawah ini, kode gagal

In [ ]:
# # Inisialisasi tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# def predict_new_data(model, text_input, tokenizer, scaler, max_len=128):
#     """
#     Melakukan prediksi untuk data teks baru.

#     Args:
#         model: Model yang sudah dilatih dan dimuat.
#         text_input (str): Teks baru untuk diuji.
#         tokenizer: Tokenizer BERT.
#         scaler: Scaler yang digunakan selama pelatihan (untuk denormalisasi).
#         max_len (int): Panjang maksimum token.

#     Returns:
#         float: Prediksi denormalisasi (nilai asli).
#     """
#     # Tokenisasi teks
#     encoding = tokenizer(
#         text_input,
#         truncation=True,
#         max_length=max_len,
#         padding='max_length',
#         return_tensors='pt'
#     )

#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     # Model inferensi
#     with torch.no_grad():
#         output = model(input_ids, attention_mask)
#         pred = output.squeeze().cpu().numpy()  # Hasil prediksi pada skala normalisasi

#     # Denormalisasi hasil prediksi
#     pred_denorm = scaler.inverse_transform(np.array(pred).reshape(-1, 1)).squeeze()

#     return pred_denorm

In [ ]:
# # Contoh input baru
# new_text = "Melakukan analisis statistik dan data mining untuk mengidentifikasi pola dan tren"

# # Dummy scaler (Ganti dengan scaler asli yang digunakan selama pelatihan)
# scaler = MinMaxScaler()
# scaler.min_, scaler.scale_ = [-1], [1]  # Pastikan menggunakan parameter asli

# # Prediksi
# predicted_salary = predict_new_data(model, new_text, tokenizer, scaler)

# print(f"Prediksi gaji untuk input baru: {predicted_salary:.2f}")

Prediksi gaji untuk input baru: 1.04


In [ ]:
# scaler = MinMaxScaler()

# train_labels = scaler.fit_transform(train_labels.reshape(-1, 1)).squeeze()
# test_labels = scaler.transform(test_labels.reshape(-1, 1)).squeeze()

In [ ]:
# import numpy as np

# # Prediksi pada skala normalisasi
# predicted_normalized = 0.04

# # Denormalisasi ke nilai asli
# predicted_original = scaler.inverse_transform(np.array(predicted_normalized).reshape(-1, 1)).squeeze()

# print(f"Prediksi gaji asli: {predicted_original}")

# Fine Tuning

In [ ]:
# 4. Fine-Tuning Model
def fine_tune_model(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        # Iterasi batch
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)

            # Menghitung loss
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}')

        # Evaluasi pada test set
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs.squeeze(), labels)
                test_loss += loss.item()

        print(f'Test Loss: {test_loss / len(test_loader):.4f}')

# Fine-Tuning the model
fine_tune_model(model, train_loader, test_loader, criterion, optimizer)

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_model.pth')

Epoch 1/3, Loss: 0.0012
Test Loss: 0.0007
Epoch 2/3, Loss: 0.0013
Test Loss: 0.0007
Epoch 3/3, Loss: 0.0012
Test Loss: 0.0007
